<p style='text-align: center'><a href=https://www.biozentrum.uni-wuerzburg.de/cctb/research/supramolecular-and-cellular-simulations/>Supramolecular and Cellular Simulations</a> (Prof. Fischer)<br>Center for Computational and Theoretical Biology - CCTB<br>Faculty of Biology, University of Würzburg</p>

<p style='text-align: center'><br><br>We are looking forward to your comments and suggestions. Please send them to: <br><br></p>
    
 <p style='text-align: center'>   <a href=andreas.kuhn@uni.wuerzburg.de>andreas.kuhn@uni.wuerzburg.de</a> or <a href=sabine.fischer@uni.wuerzburg.de>sabine.fischer@uni.wuerzburg.de</a></p>

<h1><p style='text-align: center'> Introduction to Julia </p></h1>


## 9. Example Project

### Introduction 

Until now, you have learned to use different data types, to write your own functions, to import/export/analyse data and to create beautiful plots. The question that you might be asking yourself is: "How does all of this come together?"

A hands-on exercise will provide an example. You will write a simple agent-based simulation. This involves developing a small model with randomly moving agents on a grid, using all the stuff you have already learned.




### Structure

The simulation consists of agents occupying an underlying grid. Each grid cell can only be occupied by one agent and can be accessed by its x and y coordinates. In the graphic below, a 6*7 slice of a potentially much bigger grid is shown. 


<div>
<img src="Grid_visu_1.png"width="800" />
</div>

#### 10.1. Import used packages 
 
The first step in creating such a system is (as always :) ) to import the needed packages.
 
Note: Theoretically, you could import these at any point between the beginning and the point in the program where you actually use them. However, for clarity/readability, you should always do this at the beginning of your notebook/script.

In [ ]:
#if you have not installed GLMakie so far, uncomment the code here and run this cell
#using Pkg

#Pkg.add("GLMakie")

In [3]:
using CSV, DataFrames
using GLMakie,ColorSchemes, Random

In this case, the packages `CSV` and `DataFrames` are needed for transforming and saving the output data as dataframes. Additionally, the package `GLMakie` which is the gpu-power backend for `Makie` is used. It works exactly the same way as `CairoMakie`, but outputs its plots in a separate interactive window. Therefore, it is better suited for animations or interactive plots.  
The package `ColorSchemes` provides additional colormaps for `Makie`. One function from the package `random` of the Julia standard library is also used. 

 #### 10.2 Define Parameters
The second step is to set all defining parameters of the simulation. In this case, these are the size of the grid, the number of agents, the time how long the simulation should run and the configuration/geometry in which the agents should be placed on the grid.



In [4]:
gridsize = 100                                
cell_number = 200                          
timesteps = 300                                
#starting_config = "square"
starting_config = "random"

"random"

Note: As the background of the authors is bioinformatics, we arbitrarily  declare the agents to be representing cells. But an agent can represent almost anything, especially when the only implemented rule is a random walk which is very similar for a lot of systems (atoms on lattice, molecules in a solution, stars in galaxys, insects on the ground, humans in a pedestrian zone,... ).     

 #### 10.3  Initialize system
The next step is to create the objects of the simulation. Ideally, these should only depend on the defined parameters. Here, the grid and the cells/agents have to be created. 

##### 10.3.1 Structure of data

The empty square grid can be created with the parameter gridsize and the `zeros()` function. A zero at a given gridpoint means that it is unoccupied. 

In [5]:
grid_test = zeros(Int64, gridsize, gridsize)

100×100 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0   

The system has to be populated with cells. In this case, a twofold approach is used. The positions of the cells is saved implicitly in the grid and explicitly in an additional vector. You will later see why.

To avoid slow `Any` arrays, it is best to explicitly define the type of every array used in the simulation. There are two different ways to initialize an array with a specific type. You have already seen the `type[]` constructor in the Datatype notebook. But there exists an even more powerful syntax to define arrays:

- `Array{type}(undef,dims)` where dims gives the length of the array in each dimension, e.g. dims = 10 creates a 10 elements long 1D array, dims = 5,5,5 creates a 5$\times$5$\times$5 3D-array (sometime called tensor) 

 The big advantage of this approach is that it can be nested within itself to create arbitrarily complex data structures.

Just to give you a small example how the constructor works: 

In [6]:
hans = Array{Int64}(undef,0)
dieter = Int64[]
#test if hans and dieter represent the same data structure
println(hans == dieter)

true


Note: The `undef` keyword says that the content is not yet defined. In the case of an array with length zero, this does not matter. But for a length > 0, the values in the created array will assume the unchanged bytes in the assigned memory location. For arrays consisting of primitve dataytpes like `Int64` or `Float64`, the values will be random. More complex datatypes (`dicts, arrays,...)` need a certain structure in the underlying memory. That means that the bytes can, but not necessarily need to, be a valid representation of said datatype and might be completely unusable (and cause errors) until they get overwritten.   

As the single cells are solely represented by their coordinates, a suitable data structure is a vector with `length = cellnumber` containing (sub)-vectors in the format `[x,y]`. Such a vector can be created as follows:

In [7]:
# vector of Int64-vectors with size 0
cell_list_test = Vector{Vector{Int64}}(undef,0)

Vector{Int64}[]

Sub vectors can be added to the cell_list with the `push!()` function.  

In [8]:
push!(cell_list_test,[2,3])

1-element Vector{Vector{Int64}}:
 [2, 3]

In [9]:
push!(cell_list_test,[65,12])

2-element Vector{Vector{Int64}}:
 [2, 3]
 [65, 12]

Let's remove the 2 example positions to start with the real positions: 

In [10]:
pop!(cell_list_test)
pop!(cell_list_test)
cell_list_test

Vector{Int64}[]

Note: This approach is the most performant, but also the most complicated way of setting up your data. If speed is not a concern, you can also make use of the `[]` constructor, which creates an `Any` array. Such arrays can do everything (and much more ;) ) as the one we are using, with the price of being slower. But as Julia always claims to be a fast language, we should not intentionally handicap in that regard. 

##### 10.3.2 Create cells 

When the cells are created, they have to be stored in the grid and the cell_list. In the case of a random config, the values of `x` and `y` are assigned randomly. But as two cells cannot occupy the same grid point, it is necessary to check if the assigned grid point is already occupied.

In [11]:
if starting_config == "random"
    i = 1
    while i <= cell_number
        x = rand(1:gridsize)
        y = rand(1:gridsize)
        # check if grid point is already occupied and only if it is empty 
        # create a cell and increase the counter by one 
        if grid_test[x,y] == 0 
            grid_test[x,y] = i
            push!(cell_list_test,[x,y])
            i += 1
        end
    end
end

If the starting config is a square, a bit more work has to be done. To simplify things, the cell number is rounded to the next square number. After that, the square is filled column by column with two for-loops.   

In [12]:
if starting_config == "square"
    #calculate edge length of square of cells
    edge_length = round(Int64,sqrt(cell_number))
    # round cell_number to closest square number
    cell_number = edge_length^2
    #calculate starting x and y value of corner of square on grid
    x_corner = round(Int64,gridsize/2-edge_length/2)
    y_corner = round(Int64,gridsize/2-edge_length/2)
    #all the rounding is needed to get to discrete grid points
    i = 1
    for x = 1:edge_length
        for y = 1:edge_length
            grid_test[x_corner+x, y_corner+y] = i
            push!(cell_list_test,[x_corner+x,y_corner+y])
            i += 1
        end
    end
end

In order to make things clearer, it is good practice to encapsulate different parts of your program into functions. In this case, two functions are used. One function to create and return the grid and cell_list and one to populate them according to the given starting configuration: 

In [13]:
function create_objects(Gridsize, Cell_number)
    Grid = zeros(Int64, Gridsize, Gridsize)
    Cell_list = Vector{Vector{Int64}}(undef,0)
    return Grid, Cell_list
end

function populate_sys!(Grid, Cell_list, Starting_config,Gridsize, Cell_number) 
    if Starting_config == "random"
        i = 1
        while i <= Cell_number
            x = rand(1:Gridsize)
            y = rand(1:Gridsize)
            if Grid[x,y] == 0 
                Grid[x,y] = i
                push!(Cell_list,[x,y])
                i += 1
            end
        end
    end
    if Starting_config == "square"
        #calculate edge length of square of cells
        Edge_length = round(Int64,sqrt(Cell_number))
        # round cell_number to closest square number
        Cell_number = Edge_length^2
        #calculate starting x and y value of corner of square on grid
        X_corner = round(Int64,Gridsize/2-Edge_length/2)
        Y_corner = round(Int64,Gridsize/2-Edge_length/2)
        #all the rounding is needed to get to discrete grid points
        i = 1
        for x = 1:Edge_length
            for y = 1:Edge_length
                Grid[X_corner+x, Y_corner+y] = i
                push!(Cell_list,[X_corner+x,Y_corner+y])
                i += 1
            end
        end
    end
    #return everything that could have changed
    return Grid, Cell_list, Cell_number
end

populate_sys! (generic function with 1 method)

To make it clear that the local variables inside the functions are different from the ones previously used in the global scope, they start with capital letters (but would not need to ;) ). 

 #### 10.4  Plot starting config
 Now let's execute the functions and plot the starting config to see if everything is working as intended.

In [14]:
grid, cell_list = create_objects(gridsize, cell_number)
grid, cell_list, cell_number = populate_sys!(grid, cell_list, "square",gridsize, cell_number);

This time, the plotting part is directly encapsulated into a function. You will later see why. 

In [15]:
function plot_sim(Cell_list, Gridsize)
    # make plot look nicer in black ; )
    set_theme!(theme_black())

    # using array comprehensions to create positions array of x and y
    x = [i[1] for i in Cell_list]
    y = [i[2] for i in Cell_list]
    # calculate the distance r for every cell from the center
    
    ## explain better 
    r = (((x.-Gridsize/2).^2+(y.-Gridsize/2).^2).^(1/2))

    Fig1 = Figure(resolution = (1000,1000))
    Ax1 =Axis(Fig1[1,1],title = "Startconfig",titlesize = 35)
    xlims!(Ax1,0,Gridsize)
    ylims!(Ax1,0,Gridsize)
    # using the distance r together with a colormap to give the cells different colors
    Scatty = scatter!(Ax1,x,y,color = r,colormap = :dense,label = "particle",marker = :circle,markersize = 8)

    return Fig1, Ax1 , Scatty
end

plot_sim (generic function with 1 method)

In [16]:
fig1, ax1, scatty = plot_sim(cell_list,gridsize)
display(fig1)

GLMakie.Screen(...)

A separate Makie window should open, where the plot is displayed. 

Hint: It can be annoying that the Makie window does not automatically appear on top of the other windows when creating a new plot. On Windows, the tool [pinwin](https://sourceforge.net/projects/pinwin/) can be used to permanently put Makie (or any other window) on the top of all used windows.

#### 10.5 Update function
The next step in a simulation is to write an update function. This function evolves the simulation from its starting configuration through time in discrete intervals. 

It defines the basic rules/interactions of the simulatuion as well as the time discretization.

<div>
<img src="Grid_visu_2.png"width="800" /> 
</div>
    
In each time step, every cell randomly picks one of the eight possible movement vectors to a neighbouring grid site.   
</div>
<img src="Grid_visu_3.png"width="800" />
</div>
If the selected neighbouring grid site is empty, the cell moves to that grid site and its position gets updated in the grid and the cell list.

</div>
<img src="Grid_visu_4.png"width="800" />
</div>

This process is repeated for every cell in every timestep: 

In [30]:
function update_sys(Grid, Cell_list, Gridsize,Timesteps)                  # input arguments, are the starting configuration (Grid & Cell_list) and the Gridsize and number of Timesteps
    Cell_list_alltime =  Vector{Vector{Vector{Int64}}}(undef,0)           # super cell list which contains one cell_list per time step 
    mov_vec = [[0,1],[0,-1],[1,0],[-1,0],[1,1],[-1,-1],[-1,1],[1,-1]]     # possible movement vectors to the next grid point
    sequence_vec = collect(1:8)                                           # sequence_vec is used to access the mov_vec
    
    
    Cell_list_copy = deepcopy(Cell_list)                                  # make real copy (no reference) of Cell_list and push it into cell_list_alltime 
    push!(Cell_list_alltime,Cell_list_copy)
    for t = 1:Timesteps                                                   # iterate over all timesteps
        for (j,Cell) in  enumerate(Cell_list)                             # iterate over all cells
            shuffle!(sequence_vec)                                        # use shuffle function from random to ensure random movement of particles
            for sque in sequence_vec
                x_next = Cell[1] + mov_vec[sque][1]
                y_next = Cell[2] + mov_vec[sque][2]
                
                if x_next >= 1 && x_next <= Gridsize && y_next >= 1 && y_next <= Gridsize    # check if grid point is out of bounds of grid
                    if Grid[x_next,y_next] == 0 && Grid[Cell[1],Cell[2]] != 0                # check if target grid point is empty and sanity check if cell exists on previous grid point
                        Grid[Cell[1],Cell[2]] = 0 
                        Grid[x_next,y_next]  = j 
                        
                        Cell[1] = x_next
                        Cell[2] = y_next
                        break
                    end
                end                                                                          
            end                                                                              # if no grid point fullfils the condition, the cell does not move
        end
        Cell_list_copy = deepcopy(Cell_list)                                   # creating copy of cell list in order to avoid a pass by reference (see below for an explanation of this term). 
        push!(Cell_list_alltime,Cell_list_copy)
    end
    return(Cell_list_alltime)
end

update_sys (generic function with 1 method)

If you had a hard time following the function above, here is a short summary: 

Firstly, it creates an array called `Cell_list_alltime` which is used to save a copy of the cell list in every time step.

Then, three nested loops are executed. The first one runs over all time steps. 

The second one runs over all cells (once per time step). 

The third one goes over all neighboring grid points for every cell in random order with the help of the `mov_vec[]`, `sequence_vec[]` and the `shuffle!()` function. If a neighboring grid point is empty, a cell moves there, the grid and cell_list get updated and the innermost loop breaks. If there is no empty grid point, nothing happens. 

#### Short interlude - Pass by value or by reference 

You may also wonder why the `deepcopy()` function is used before the cell_list is `push!()` into the `Cell_list_alltime`. This is due to a concept that has not been introduced yet: [passing by reference](https://stackoverflow.com/questions/38936868/in-julia-functions-passed-by-reference-or-value). Essentially, it means that all mutable datatype (like arrays, dicts,...) are not copied when they are assigned to a new variable. The new variable just points to the same already existing object in memory. See small example below:   

In [19]:
dieter = [2,5,34]
dieter2 = dieter
dieter2[2] = 7777
#even though we haven't changed dieter, it got modified as dieter2 is only a reference pointing to the same object
println(dieter, dieter === dieter2) 
# the === operator checks if two variables are refering to the same object

[2, 7777, 34]true


In [20]:

dieter3 = [2,5,34]
dieter4 = deepcopy(dieter3)
dieter4[2] = 7777
#now dieter4 is a new object 
println(dieter3, dieter3 === dieter4)

[2, 5, 34]false


#### Interlude end

Let's execute the `update_sys` function and plot the cell positions in the last time step. 

In [31]:
cell_all = update_sys(grid,cell_list,gridsize,timesteps)

fig2, ax2, scatty2 = plot_sim(cell_all[end],gridsize)
display(fig2)

GLMakie.Screen(...)

The cells have moved! For a more complex analysis, it's useful to convert the `cell_all` array into a data frame.  

You might be wondering why the simulation does not run directly on `DataFrames` instead of arrays. It is possible to do that and in many cases it would not make a big difference, but `DataFrames` are slower and use more memory. Therefore, if a simulation needs to be scalable to big cell numbers and/or time steps, arrays are a better choice.

In [22]:
function convert_to_DF(cell_list_alltime)
    cell_number = length(cell_list_alltime[1])
    Data = DataFrame(timestep = ones(Int64,cell_number),id= collect(1:cell_number),
        x = [i[1] for i in cell_list_alltime[1]],y = [i[2] for i in cell_list_alltime[1]])
    for j in 2:length(cell_list_alltime)
        append!(Data,DataFrame(timestep = ones(Int64,cell_number).*j,id= collect(1:cell_number),
                x = [i[1] for i in cell_list_alltime[j]],y = [i[2] for i in cell_list_alltime[j]]))
    end
    return Data
end

convert_to_DF (generic function with 1 method)

In [23]:
length(cell_all)

301

In [24]:
data = convert_to_DF(cell_all)

Row,timestep,id,x,y
,Int64,Int64,Int64,Int64
1,1,1,44,44
2,1,2,44,45
3,1,3,44,46
4,1,4,44,47
5,1,5,44,48
6,1,6,44,49
7,1,7,44,50
8,1,8,44,51
9,1,9,44,52


As the simulation data is now in form of a `DataFrame`, many of the `DataFrame` specific tools from the data_analysis notebook can be used. But first things first, let's save our data with the `CSV` package, just to be sure. 

In [25]:
CSV.write("$(gridsize)_positions.csv",data)

"100_positions.csv"

It is good practice to save a settings file which includes all the parameters of the simulation together with your raw data. The easiest way to do that is a `dict`. 

In [26]:
function save_settings(Gridsize,Cell_number,Timesteps,Starting_config)
    settings = Dict("gridsize" => Gridsize, "cell_number" => Cell_number, 
        "timesteps" =>Timesteps, "starting_config" => Starting_config)
    open("settings.txt","w") do file
        print(file,settings)
    end 
end

save_settings (generic function with 1 method)

 ### 10.6 Putting everything together
 Because all the simulation steps are encapsulated in functions, it is possible to execute a whole simulation in one cell. This time with the "square" starting config. 

In [28]:
gridsize = 200                               
cell_number = 1000                         
timesteps = 200                           
starting_config = "square"

grid, cell_list = create_objects(gridsize, cell_number)
grid, cell_list, cell_number = populate_sys!(grid, cell_list, starting_config,gridsize, cell_number)
cell_all_2 = update_sys(grid,cell_list,gridsize,timesteps)
data2 = convert_to_DF(cell_all_2)
CSV.write("$(gridsize)_positions.csv",data2)
save_settings(gridsize,cell_number,timesteps,starting_config)
fig3,ax3, scatty3 =plot_sim(cell_all_2[end],gridsize)
display(fig3)

GLMakie.Screen(...)

### 10.7 Outlook: Let's make a beautiful animation :=)
You do not need to understand everything in this section, as concepts are used that have not been introduced yet. This should serve the purpose of showing you what is possible in Julia and also give you a very nice looking finish of this course. So sit back and enjoy ;).  
If you are interested in understanding what is going on here: Watch this [video](https://www.youtube.com/watch?v=L-gyDvhjzGQ ) of the brilliant Julia developer George Datseris, where he explains all the concepts needed to understand the code below. 

#### Short summary: 
Two functions are created here, one for a 2D animation and one for a 3D animation. Both need a grouped dataframe as argument together with the gridsize. Both functions return a figure object and a variable called t which represents the timesteps and is an object of type `Observable`. This `Observable` has the special ability, that all objects that depend on it (in this case x,y,r,titel and the figure) are automatically updated when it changes. Therefore, the `run_animation` function for both 2D and 3D is the same and just slowly increases the timesteps t. The plot then automatically updates :).

Note: Compared to other plotting packages like `matplotlib` in Python or `ggplot` in R, this is a very different way to create animations or videos. The big advantage of the `Observable` based approach is that you do not have to define a new plot object for every frame, you just need to specify what has changed between frames. Together with the gpu accelaration, this allows a much better performance.  

In [32]:
function animation_2D(Data_gr,Gridsize) 
    t = Observable(1)
    x = @lift(Data_gr[$t].:x)                                          # @lift macro is needed when creating Observables which depend on other Observables
    y = @lift(Data_gr[$t].:y) 
    r = @lift((($x.-Gridsize/2).^2+($y.-Gridsize/2).^2).^(1/2))
    titel = @lift(string($t))

    set_theme!(theme_black())

    fig6 = Figure(resolution = (1000,1000))
    ax6 =Axis(fig6[1,1],title = @lift("timestep : $(round(Int64,$t))"),titlesize = 35)
    xlims!(ax6,0,Gridsize)
    ylims!(ax6,0,Gridsize)
    scatty6 = scatter!(ax6,x,y,color = r,colormap = :dense,label = "particle",marker = :circle,markersize = 15)

    #hidedecorations!(ax6)
    axislegend(ax6)
    display(fig6)
    return t,fig6
end

animation_2D (generic function with 1 method)

In [33]:
function animation_3D(Data_gr,Gridsize) 
    
    # make sure that camera flight is not too fast
    if length(Data_gr) >= 1000
        len = length(Data_gr)
    else
        len = 1000
    end
    t = Observable(1)
    
    x = @lift(Data_gr[$t].:x)
    y = @lift(Data_gr[$t].:y)
    r = @lift((($x.-Gridsize/2).^2+($y.-Gridsize/2).^2).^(1/2))
    titel = @lift(string($t))
   
    # define the camera angle
    elevations = range(start = -2π,stop = 2π, length = len)
    azimuths = range(start = 0,stop = 2π, length = len)
    z = zeros(length(Data_gr[1].:x))
    set_theme!(theme_black())

    fig7 = Figure(resolution = (2000,2000))
    ax7 =Axis3(fig7[1,1],
        title = @lift("timestep : $(round(Int64,$t))"),
        elevation =@lift(elevations[$t])  , azimuth = @lift(azimuths[$t]),
        viewmode = :fit,
        titlesize = 35,protrusions = (0, 0, 0, 40)
        )   

    xlims!(ax7,0,Gridsize)
    ylims!(ax7,0,Gridsize)
    zlims!(-Gridsize/2,Gridsize/2)

    scatty7 = scatty = meshscatter!(ax7,x,y,z,markersize = 2.5,color = r,
        colormap = :diverging_gkr_60_10_c40_n256
        ,label = "particle")
    #hidedecorations!(ax7)
    display(fig7)
    return t,fig7
end

animation_3D (generic function with 1 method)

Additonally, a function for running the animation and one for running and saving the animation into a video, have to be defined. As all the plotting features have already been specified in the `animation_2D()` respectively `animation_3D()`, these functions only increment the `Observable` `t` by 1 in every frame. 

In [34]:
function run_animation(T,Timesteps)
    @async for i in 2:1:Timesteps
        T[] = T[]+1
        sleep(0.05)
    end
    T[] = 1
end

run_animation (generic function with 1 method)

In [35]:
function save_animation(T,Fig,Data_gr)
    record(Fig, "beautiful2.mp4",1:length(Data_gr); framerate = 30) do i
    T[] = i
    end
end

save_animation (generic function with 1 method)

Comment: You might be thinking that animating a 2D system in 3D is quite stupid and unneccessary. And you are right. From a scientifc perspective, that does not make much sense. But in this case, it serves other purposes and sh:  

1. 2D and 3D plots/animations are quite similar (only add z-coordinate and define camera)
2. They can be updated, handled and saved with the same functions  
3. 3D is beautiful ;) 

Lets create some data to run an animation. 

In [36]:
data_gr= groupby(data2,:timestep);

In [37]:
t,fig_2d = animation_2D(data_gr,gridsize);

Run the 2D animation:

In [38]:
run_animation(t,timesteps)

1

Create a 3D animation: 

In [39]:
t3,fig_3d = animation_3D(data_gr,gridsize);

Run the 2D animation:

In [40]:
run_animation(t3,timesteps)

1

## Exercises

### <p style='color: green'>easy</p>

1. Make some simulations for very small and very large systems and admire the beautiful animations.

2. The color scheme used in the 3D animation is not very friendly to people with a red-green deficiency. Change that to a more friendly color scheme. If you don not remember where to find a list of the available color schemes, take a second look at the plotting chapter of this course. 

3. Use the `save_animation()` function to save your favorite simulation as a video.  

### <p style='color: orange'>medium</p>

4. Define a new parameter `name` which should be part of the name of all created files of the simulation (csv, settingsfile, video). Change all involved functions accordingly. 

5. Normally, you would not put such long function definitions into a jupyter notebook and then use them in the same jupyter notebook below. Typically, you would put them into an external file and import them at the beginning. Take a look at how `include()` [works](https://docs.julialang.org/en/v1/manual/code-loading/). Create a `test.jl` file with an example function written by you and include it into a jupyter notebook and execute it there.   

6. Put all the functions defined here into a `random_walk.jl` file. Include the file into a new juypter notebook called `executor.ipynb` and run the simulation and animation there. 


7. Change the movement of the cells by changing the `update_sys()` function to `update_sys_bias()` in `random_walk.jl`. Remove one of the possible movement vectors on the grid to give them a bias in a certain direction. How does the result change for long time spans? Save a video of your changed simulation. 

Hint: If you are changing a `.jl` you have to include it again, so that the changes are active in the Julia session. 

### <p style='color: red'>hard</p>


8. The starting configuration in a square is not very realistic, add another option in the `populate_sys!` that puts the cells into a circle. 

### <p style='color: red'>Very hard - Only do when you want to punish yourself</p>

9. A typical way to quantify a random walk is the mean squared displacement [(MSD)](https://en.wikipedia.org/wiki/Mean_squared_displacement). Make a simulation run with the following parameters: 
```julia
gridsize = 1000                               
cell_number = 1000                         
timesteps = 5000                           
starting_config = "random"
name = "msd"
```
Use the created dataframe to calculate the MSD. Plot the MSD against time in a line plot and save it. What behaviour do you see and what is expected from a [random walk](https://en.wikipedia.org/wiki/Anomalous_diffusion)? 